In [1]:
import pyiqfeed as iq
import pandas as pd
import datetime
import redis

In [2]:
redis_host = "127.0.0.1" #'dt-tor-03422.cppib.ca' 
redis_port = 6379
redis_db = 0
redis_server = redis.StrictRedis(host=redis_host, port=redis_port, db=redis_db, ssl=False)

In [3]:
ASSET_NAME = "name"
DATE_NAME = "date"
TIME_NAME = "time"
DATETIME_NAME = "datetime"
OPEN_NAME = "open"
HIGH_NAME = "high"
LOW_NAME = "low"
CLOSE_NAME = "close"
VOLUME_NAME = "volume"
VOLUME_TOTAL_NAME = "volume_total"
OPEN_INTEREST_NAME = "open_interest"
TRADE_NUM_NAME = "trade_number"

DICT_NAME_MAPPER = {
    "date": DATE_NAME,
    "open_p": OPEN_NAME,
    "high_p": HIGH_NAME,
    "low_p": LOW_NAME,
    "close_p": CLOSE_NAME,
    "prd_vlm": VOLUME_NAME,
    "open_int": OPEN_INTEREST_NAME,
    "tot_vlm": VOLUME_TOTAL_NAME,
    "num_trds": TRADE_NUM_NAME,
    "time": TIME_NAME,
    "last_sz": VOLUME_NAME,
    "last": CLOSE_NAME,
}

In [4]:
def launch_service(dtn_login, dtn_password, dtn_product_id):
    """Check if IQFeed.exe is running and start if not"""

    svc = iq.FeedService(
        product=dtn_product_id,
        version="Debugging",
        login=dtn_login,
        password=dtn_password,
    )
    svc.launch(headless=False)
from localconfig.passwords import dtn_login, dtn_password, dtn_product_id

launch_service(dtn_login, dtn_password, dtn_product_id)

In [6]:
end_date = datetime.datetime.today()
start_date = datetime.datetime.today() - pd.offsets.BDay(50)

# list_tickers = redis_server.keys()
list_tickers = pd.read_clipboard()["Ticker"].tolist()

list_tickers = set(list_tickers)
# list_tickers = [l.decode("utf-8") for l in list_tickers]

In [7]:
hist_conn = iq.HistoryConn(name="hist-conn")
hist_listener = iq.VerboseIQFeedListener("History Bar Listener")
hist_conn.add_listener(hist_listener)

dict_result = {}

with iq.ConnConnector([hist_conn]) as connector:
    for ticker in list_tickers:
        try:
            # print(f"Processing {ticker}")
            d_start_date = pd.to_datetime(start_date).date()
            d_end_date = pd.to_datetime(end_date).date()
            daily_data = hist_conn.request_daily_data_for_dates(ticker, d_start_date, d_end_date, ascend=True)
            df_daily = pd.DataFrame(daily_data).rename(columns=DICT_NAME_MAPPER).set_index(DATE_NAME)
            if ".FXCM" in ticker:
                ticker = ticker.replace(".FXCM", "")
            dict_result[ticker] = df_daily
        except (iq.NoDataError, iq.UnauthorizedError) as err:
            print(f"Wrong Data {ticker}")

Wrong Data WWAC
Wrong Data LSI
Wrong Data RACY
Wrong Data PYR
Wrong Data VEDU
Wrong Data DKDCA
Wrong Data LJIM
Wrong Data UNVR
Wrong Data BLU
Wrong Data CLTL
Wrong Data DICE
Wrong Data AFTR
Wrong Data EMZA
Wrong Data PSJ
Wrong Data VBLT
Wrong Data ISEE
Wrong Data SGTX
Wrong Data RMED
Wrong Data LENB
Wrong Data DNAB
Wrong Data SFR
Wrong Data HYLD
Wrong Data VALT
Wrong Data ILLM
Wrong Data MTVR
Wrong Data XPAX
Wrong Data OFC
Wrong Data TAPA
Wrong Data GFX
Wrong Data WTT
Wrong Data BIOC
Wrong Data SCAQ
Wrong Data PSDN
Wrong Data PIAI
Wrong Data VHNA
Wrong Data AKOA
Wrong Data MOGB
Wrong Data CORS
Wrong Data ATTO
Wrong Data NUVA
Wrong Data SVA
Wrong Data CWENA
Wrong Data VACC
Wrong Data NEX
Wrong Data WE
Wrong Data SLGG
Wrong Data AKOB
Wrong Data IBET
Wrong Data IDBA
Wrong Data FORG
Wrong Data TLGA
Wrong Data EGLX
Wrong Data QUOT
Wrong Data PBS
Wrong Data SURF
Wrong Data JRO
Wrong Data BVXV
Wrong Data PYPT
Wrong Data RAD
Wrong Data GRIL
Wrong Data BWC
Wrong Data GTNA
Wrong Data WWE
Wrong D

In [8]:
list_dfs_low = []
list_dfs_high = []
for ticker in dict_result.keys():
    list_dfs_high.append(dict_result[ticker][HIGH_NAME].to_frame(ticker))
    list_dfs_low.append(dict_result[ticker][LOW_NAME].to_frame(ticker))
df_high = pd.concat(list_dfs_high, axis=1)
df_low = pd.concat(list_dfs_low, axis=1)

In [9]:
s = (df_high - df_low).tail(30).mean() + (df_high - df_low).tail(30).std()

In [10]:
for ticker in s.index:
    redis_server[ticker] = s[ticker].round(5)